# Relationship to MUA
<p>Figures: schematic MUA, epoch-based linear regression of correlation with aperiodic parameters, temporal correlation
<p>
<p>created: Septermber 18, 2024
<p>last modified: September 23, 2024


## Set-up

In [1]:
# auto-reload
%load_ext autoreload
%autoreload 2

In [2]:
# general
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from scipy.stats import zscore

# custom
import sys
sys.path.append("../../code")
from paths import EXTERNAL_PATH
from info import SESSIONS, FS, TOTAL_CHANS, EPOCH_TIMES
from plots import plot_schematic, plot_regression
from utils import compute_confidence_interval, subtract_baseline

### Settings

In [13]:
# set plotting style
plt.style.use('../../mpl_styles/sfn_2024.mplstyle')

# choose seesion to visualize
session = SESSIONS[1]


### Create output directory

In [4]:
dir_out = f"{EXTERNAL_PATH}/figures/sfn_2024"
if not os.path.exists(dir_out):
    os.makedirs(dir_out)

#### load data

In [ ]:
# load MUA data
mua = pd.read_csv(fr"{EXTERNAL_PATH}\data\results\mua_df.csv")
mua_pre = mua.loc[(mua['session'] == session) &
                (mua['epoch'] == 'pre'), 'mua']
mua_post = mua.loc[(mua['session'] == session) &
                (mua['epoch'] == 'post'), 'mua']
mua_diff = mua_post.values - mua_pre.values

mua

In [ ]:
# load spectral parameters and merge with mua
params = pd.read_csv(fR"{EXTERNAL_PATH}\data\results\lfp_spectral_params.csv", index_col=0)
df = pd.merge(params, mua, on=['session', 'array', 'epoch', 'channel'])
df

In [7]:
# load time-resolved parameters and MUA, and pre-process for plotting
data_mua = pd.read_csv(f"{EXTERNAL_PATH}/data/results/lfp_stm_params_mua.csv")
session_df_mua = data_mua[data_mua.get('session') == SESSIONS[0]].loc[((data_mua['window']>124) & (data_mua['window']<375))]
session_df_mua['time'] = (session_df_mua.get('window')/FS) + EPOCH_TIMES[0]

vars = ['offset', 'exponent', 'mua']
ci = {'lci_offset':[], 'rci_offset':[], 'lci_exponent':[], 'rci_exponent':[], 'lci_mua':[], 'rci_mua':[]}

for var in vars:
    print(f'starting var : {var}')
    count = 0
    z_vars = np.array([])

    for i in range(TOTAL_CHANS):
        # channel zscore values
        data_a = session_df_mua.iloc[np.arange(count, count + 250)]

        z_var = zscore(data_a[var], nan_policy='omit')
        z_vars = np.append(z_vars, z_var)

        count = count + 250

    # add zscore and time variables to df
    session_df_mua[f"z_{var}"] = z_vars
    
    # subtract baseline
    print('subtracting baseline')
    session_df_mua[f'relative_{var}'] = subtract_baseline(session_df_mua.get(f'z_{var}'), session_df_mua.get('time'), np.array([session_df_mua.get('time').min(), 0]))

    # compute left and right endpoint confidence intervals
    lci_var = np.array([])
    rci_var = np.array([])

    for i in np.unique(session_df_mua.get('window')):
        window_df = session_df_mua[session_df_mua.get('window') == i]

        ci_var = compute_confidence_interval(window_df.get(f'relative_{var}'))
        ci[f'lci_{var}'].append(ci_var[0])
        ci[f'rci_{var}'].append(ci_var[1])
        
# average components per window
session_df_mua = session_df_mua.groupby('window').mean().reset_index()
session_df_mua

## Main

In [ ]:
# create figure and gridspec
fig = plt.figure(figsize=[18, 10], constrained_layout=True)
spec = gridspec.GridSpec(figure=fig, ncols=2, nrows=2, width_ratios=[1,1], 
                         height_ratios=[3/4,1])

# create subplots (first row)
ax_a = fig.add_subplot(spec[0,0])
ax_b = fig.add_subplot(spec[0,1])

# create nested gridspec in second row
spec_a = spec[1,:].subgridspec(ncols=3, nrows=1, width_ratios=[1,1,1])
ax_c = fig.add_subplot(spec_a[0,0])
ax_d = fig.add_subplot(spec_a[0,1])
ax_e = fig.add_subplot(spec_a[0,2])

# set PLACEHOLDER titles
ax_a.set_title("Post-stimulus MUA (schematic)")
ax_b.set_title("Difference in MUA (schematic)")
ax_c.set_title("Exponent v MUA")
ax_d.set_title("Offset v MUA)")
ax_e.set_title("Temporal correlation")

# plot subplot a-b: schematic on MUA -------------------------------------------
odml_path = f"{EXTERNAL_PATH}/V1_v4_1024_electrode_resting_state_data/data/{session}/metadata_{session}.odml"
plot_schematic(mua_post, odml_path, ax=ax_a)
plot_schematic(mua_diff, odml_path, norm_type='centered', ax=ax_b)

# plot subplot c-d: epoch-based correlation ------------------------------------
for var, ax_lin in zip(['offset', 'exponent'], [ax_c, ax_d]):
    for i_session, session in enumerate(SESSIONS):
        x_data = df.loc[(df['session']==session) & (df['epoch']=='post'), 'mua']
        y_data = df.loc[(df['session']==session) & (df['epoch']=='post'), var]

        plot_regression(x_data, y_data, ax=ax_lin, xlabel='log(MUA)', 
                        ylabel=f'LFP {var}', label=session, 
                        label_offset=0.2*i_session)

# plot subplot e: temporal correlation -----------------------------------------
ax_e.plot(session_df_mua.get('time'), session_df_mua.get('relative_offset'), c='#2C73D2', label='offset')
ax_e.fill_between(session_df_mua.get('time'), ci.get('lci_offset'), ci.get('rci_offset'), color='#2C73D2', alpha=0.4)

ax_e.plot(session_df_mua.get('time'), session_df_mua.get('relative_exponent'), c='#00C9A7', label='exponent')
ax_e.fill_between(session_df_mua.get('time'), ci.get('lci_exponent'), ci.get('rci_exponent'),  color='#00C9A7', alpha=0.4)

ax_e.plot(session_df_mua.get('time'), session_df_mua.get('relative_mua'), c='#845EC2', label='mua')
ax_e.fill_between(session_df_mua.get('time'), ci.get('lci_mua'), ci.get('rci_mua'),  color='#845EC2', alpha=0.4)
ax_e.set_xlabel('Time (s)')
ax_e.set_ylabel(r'$\Delta$ Z-score')
ax_e.axhline(y=0, color='k', linestyle='--')
ax_e.axvline(x=0, color='k', linestyle='--')

# save figure
plt.savefig(f'{dir_out}/panel_7.png')